# Mini Project 2

The purpose of this project is classification using machine learning.

The following section is imitialization of all dependencies or libraries.

In [6]:
from tensorflow.keras import (
    Input,
    Model
)

from tensorflow.keras.layers import (
    Conv2D,
    MaxPool2D,
    ReLU,
    BatchNormalization,
    GlobalAveragePooling2D,
    Flatten,
    Dense,
    Add,
    Softmax
    )

from tensorflow.keras import cifar10
import numpy as pn
import os 

import tarfile




Some comment here

In [7]:
def _residual_block_identity(inputs,filters,strides=1): # bottleneck approach
    # padding ='same' is important to keep the size of the feature maps the same
    # This way the ouput size only depends on the number of filters used
    x = Conv2D(filters=filters,kernel_size=(1,1), strides=strides,padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=filters,kernel_size=(3,3), strides=1,padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=(filters * 4),kernel_size=(1,1), strides=1,padding='same')(x)
    x = BatchNormalization()(x)

    res = Add()([x,inputs])
    return x

In [8]:
def _residual_block_shortcut(inputs,filters,strides=1): # bottleneck approach
    # padding ='same' is important to keep the size of the feature maps the same
    # This way the ouput size only depends on the number of filters used
    x = Conv2D(filters=filters,kernel_size=(1,1), strides=strides,padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=filters,kernel_size=(3,3), strides=1,padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=(filters * 4),kernel_size=(1,1), strides=1,padding='same')(x)
    x = BatchNormalization()(x)

    # apply projection to make input shape match output shape
    res = Conv2D(filters=filters * 4,kernel_size=(1,1), strides=strides,padding='same')(inputs)
    res = BatchNormalization()(res)
    res = Add()([x,res])
    x = ReLU()(res)
    return x

In [9]:
def get_model(input_shape,num_class=1000):
    input_ = Input(shape=input_shape)
    #conv1
    x = Conv2D(filters=64,kernel_size=(7,7), strides=2,padding='same')(input_)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    print('conv1_x output size',x.shape)

    x = MaxPool2D(pool_size=(3,3),strides=2,padding='same')(x) 

    #conv2_x - stage 1 (3)
    x = _residual_block_shortcut(x,filters=64)
    x = _residual_block_identity(x,filters=64)
    x = _residual_block_identity(x,filters=64)
    print('conv2_x output size',x.shape)

    #conv3_x - stage 2 (4)
    x = _residual_block_shortcut(x,filters=128,strides=2)
    x = _residual_block_identity(x,filters=128)
    x = _residual_block_identity(x,filters=128)
    x = _residual_block_identity(x,filters=128)
    print('conv3_x output size',x.shape)

    #conv4_x - stage 3 (6)
    x = _residual_block_shortcut(x,filters=256,strides=2)
    x = _residual_block_identity(x,filters=256)
    x = _residual_block_identity(x,filters=256)
    x = _residual_block_identity(x,filters=256)
    x = _residual_block_identity(x,filters=256)
    x = _residual_block_identity(x,filters=256)
    print('conv4_x output size',x.shape)

    #conv5_x - stage 4 (3)
    x = _residual_block_shortcut(x,filters=512,strides=2)
    x = _residual_block_identity(x,filters=512)
    x = _residual_block_identity(x,filters=512)
    print('conv5_x output size',x.shape)
    
    x = GlobalAveragePooling2D()(x)
    '''
    Total params: 23,587,712
    Trainable params: 23,534,592
    Non-trainable params: 53,120

    '''
    x = Flatten()(x)
    x = Dense(units=num_class)(x)
    x = Softmax()(x)
    '''
    Total params: 25,636,712
    Trainable params: 25,583,592
    Non-trainable params: 53,120
    '''
    return Model(inputs=input_,outputs=x,name='resnet_50')


In [10]:
if __name__ == '__main__':
    import os
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    model = get_model((224,224,3))
    model.summary()


conv1_x output size (None, 112, 112, 64)
conv2_x output size (None, 56, 56, 256)
conv3_x output size (None, 28, 28, 512)
conv4_x output size (None, 14, 14, 1024)
conv5_x output size (None, 7, 7, 2048)
Model: "resnet_50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
_______________________________________________________________________________________